Input

In [89]:
file_path = "input.txt"  # Path to the input file

with open(file_path, "r") as f:
    data = f.read()

codes = data.split('\n')

#### Initial Attempt: Rank 783

> Sadly the optimized approach below only works till depth 10

In [37]:
# +---+---+---+
# | 7 | 8 | 9 |
# +---+---+---+
# | 4 | 5 | 6 |
# +---+---+---+
# | 1 | 2 | 3 |
# +---+---+---+
#     | 0 | A |
#     +---+---+

numeric_keyboard = {
    '0': [('>', 'A'), ('^', '2')],
    '1': [('^', '4'), ('>', '2')],
    '2': [('<', '1'), ('>', '3'), ('^', '5'), ('v', '0')],
    '3': [('^', '6'), ('<', '2'), ('v', 'A')],
    '4': [('^', '7'), ('v', '1'), ('>', '5')],
    '5': [('<', '4'), ('>', '6'), ('^', '8'), ('v', '2')],
    '6': [('^', '9'), ('<', '5'), ('v', '3')],
    '7': [('>', '8'), ('v', '4')],
    '8': [('v', '5'), ('<', '7'), ('>', '9')],
    '9': [('<', '8'), ('v', '6')],
    'A': [('^', '3'), ('<', '0')]
}

#     +---+---+
#     | ^ | A |
# +---+---+---+
# | < | v | > |
# +---+---+---+
direction_keyboard = {
    '^': [('v', 'v'), ('>', 'A')],
    'A': [('v', '>'), ('<', '^')],
    '<': [('>', 'v')],
    'v': [('^', '^'), ('<', '<'), ('>', '>')],
    '>': [('^', 'A'), ('<', 'v')],
}

In [38]:
from collections import deque

# BFS
def bfs_all_shortest_paths(keyboard, start, end):
    queue = deque([(start, "")])  # (current position, path)
    visited = {start: 0}  # Tracks shortest distance to each node
    paths = []

    while queue:
        current, path = queue.popleft()
        
        # If we reach the end, add path to results
        if current == end:
            paths.append(path)
            continue
        
        # Expand neighbors
        for move, neighbor in keyboard[current]:
            new_path = path + move
            if neighbor not in visited:
                visited[neighbor] = len(new_path)
                queue.append((neighbor, new_path))
            # Allow paths of equal length
            elif len(new_path) == visited[neighbor]:
                queue.append((neighbor, new_path))
    
    return paths


# Generate all-pairs shortest paths
def generate_all_shortest_paths(keyboard):
    keys = list(keyboard.keys())
    all_paths = {k: {} for k in keys}  # Nested dictionary for paths

    for start in keys:
        for end in keys:
            if start == end:
                all_paths[start][end] = [""]  # Trivial path for self-loop
            else:
                all_paths[start][end] = bfs_all_shortest_paths(keyboard, start, end)
    return all_paths

from itertools import product

def generate_combinations(paths):
    # Generate all combinations by taking one element from each sublist
    all_combinations = product(*paths)
    
    results = []
    for combo in all_combinations:
        result = ''.join(combo)
        results.append(result)
    
    return results

In [39]:
shortest_paths_directional = generate_all_shortest_paths(direction_keyboard)
ref_paths_directional = generate_all_shortest_paths(direction_keyboard)

for k in shortest_paths_directional.keys():
    for k_n in shortest_paths_directional[k]:
        if len(shortest_paths_directional[k][k_n]) == 1:
            shortest_paths_directional[k][k_n] = shortest_paths_directional[k][k_n][0]
        else:
            candidate = ""
            min_sz = 100
            for pattern in shortest_paths_directional[k][k_n]:
                patch_pattern = 'A' + pattern + 'A'
                moves_curr_sub = []
                for (x, y) in zip(patch_pattern, patch_pattern[1:]):
                    if x != y: moves_curr_sub.append(ref_paths_directional[x][y])
                    moves_curr_sub.append(['A'])
                moves_curr_sub = generate_combinations(moves_curr_sub)

                len_gen = min(len(m) for m in moves_curr_sub)
                if len_gen < min_sz: 
                    min_sz = len_gen
                    candidate = pattern
            shortest_paths_directional[k][k_n] = candidate

shortest_paths_numeric = generate_all_shortest_paths(numeric_keyboard)

for k in shortest_paths_numeric.keys():
    for k_n in shortest_paths_numeric[k]:
        if len(shortest_paths_numeric[k][k_n]) == 1: continue
        candidates = []
        min_sz = 100
        for pattern in shortest_paths_numeric[k][k_n]:
            patch_pattern = 'A' + pattern + 'A'
            moves_curr_sub = []
            for (x, y) in zip(patch_pattern, patch_pattern[1:]):
                if x != y: moves_curr_sub.append(ref_paths_directional[x][y])
                moves_curr_sub.append(['A'])
            moves_curr_sub = generate_combinations(moves_curr_sub)

            len_gen = min(len(m) for m in moves_curr_sub)
            if len_gen < min_sz: 
                min_sz = len_gen
                candidates = [pattern]
            elif len_gen == min_sz:
                candidates.append(pattern)

        shortest_paths_numeric[k][k_n] = candidates

In [ ]:
from tqdm import tqdm

def process_pattern(pattern):
    initial_key = 'A'
    patch_pattern = initial_key + pattern
    moves1 = []
    for (x, y) in zip(patch_pattern, patch_pattern[1:]):
        if x != y: moves1.append(shortest_paths_numeric[x][y])
        moves1.append(['A'])

    moves_prev = generate_combinations(moves1)
    moves_curr = []
    for i in tqdm(range(20)):
        moves_curr = []
        for moves in moves_prev:
            patch_pattern = initial_key + moves
            moves_curr_sub = ''
            for (x, y) in zip(patch_pattern, patch_pattern[1:]):
                if x != y: moves_curr_sub += shortest_paths_directional[x][y]
                moves_curr_sub += 'A'
            moves_curr.append(moves_curr_sub)
        moves_curr = list(set(moves_curr))
        moves_prev = moves_curr
        moves_curr = []
    
    digs = int(''.join([char for char in pattern if char.isdigit()]))
    return min(len(m) for m in moves_prev)*digs

sm = 0
for code in codes: sm += process_pattern(code)
sm

#### Optimized

Common

In [43]:
## new way of representing by row and column coords

# +---+---+---+
# | 7 | 8 | 9 |
# +---+---+---+
# | 4 | 5 | 6 |
# +---+---+---+
# | 1 | 2 | 3 |
# +---+---+---+
#     | 0 | A |
#     +---+---+

numeric_keypad = {
    "7": (0, 0),
    "8": (0, 1),
    "9": (0, 2),
    "4": (1, 0),
    "5": (1, 1),
    "6": (1, 2),
    "1": (2, 0),
    "2": (2, 1),
    "3": (2, 2),
    "0": (3, 1),
    "A": (3, 2)
}

#     +---+---+
#     | ^ | A |
# +---+---+---+
# | < | v | > |
# +---+---+---+
direction_keypad = {
    "^": (0, 1),
    "A": (0, 2),
    "<": (1, 0),
    "v": (1, 1),
    ">": (1, 2)
}

dd = {
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1),
    "^": (-1, 0)
}

In [93]:
import sys
from functools import lru_cache
from itertools import permutations 

@lru_cache(None)
def generate_ways(a, b, keypad): # generate ways to go from key_a to key_b on a given board
    keypad = direction_keypad if keypad else numeric_keypad

    x, y = keypad[a], keypad[b]
    di, dj = y[0] - x[0], y[1] - x[1]

    moves = ""
    if di > 0: moves += "v"*di
    else: moves += "^"*-di
    if dj > 0: moves += ">"*dj
    else: moves += "<"*-dj

    combo_moves = [''.join(x) + 'A' for x in list(set(permutations(moves)))]
    valid_moves = []

    for mv in combo_moves:
        ci, cj = x
        flag = True
        for d in mv[:-1]:
            di, dj = dd[d] 
            ci, cj = di + ci, dj + cj
            if (ci, cj) not in keypad.values(): 
                flag = False
                break
        if flag: valid_moves.append(mv)
    
    return valid_moves

@lru_cache
def keystrokes(a, b, keypad, chain):
    if chain == 0: return min([len(x) for x in generate_ways(a, b, True)])
    ways = generate_ways(a, b, keypad) # we start with numeric keypad in first iteration then move on to direction keypad
    min_strokes = sys.maxsize

    for way in ways:
        patch_way = f"A{way}" # we start with key A
        strokes = 0
        for x, y in zip(patch_way, patch_way[1:]):
            strokes += keystrokes(x, y, True, chain-1)
        min_strokes = min(min_strokes, strokes)
    
    return min_strokes


In [96]:
def code_cost(code, chain):
    patch_code = "A" + code
    strokes = 0
    for x, y in zip(patch_code, patch_code[1:]):
        strokes += keystrokes(x, y, False, chain)
    return strokes * int(code[:-1])

Part 1

In [98]:
ans = 0
for code in codes: ans += code_cost(code, 2)
ans

197560

Part 2

In [103]:
ans = 0
for code in codes: ans += code_cost(code, 25)
ans

242337182910752